In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input director
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Introduction

Dans le domaine informatique, il est n'est pas évident de savoir quel poste corresponds á un candidat, ou encore un personne à la capacité d'accomplir plusieurs responsabilités. La raison est que très souvent, un candidat en informatique possède des compétences dans de domaine varié en l'informatique. Ce dernier peux s'épanouir á un poste que si seulement les combinaisons de ces compétences reponds aux exigences du poste. Je donne un exemple, un étudiant qui a eu son diplôme en informatique, peut á la fois postuler pour un post de développeur d'application web, ou de dévéloppeur de base de données etc. La difficulté qu'on rencontre ou la question qu'on se pose est la suivant : Quel poste correspond le mieux á un candidat si on se base sur les combinaisons de ces competences techniques. Ce travail, va utilisé l'intélligence artificielle, plus précisement machine learning pour répondre á cette question.

Ci dessous, les packages que nous allons utilisés dans ce projet
numpy pour effectuer les opérations numerique, c'est d'ailleurs le package le plus utilisés
pandas le package fournissant des outils d'analyse de données, il offre une representation structuré des données, comme les series et les dataframe. Il fournie egalement des fonctions permettant de faire l'analyse de données.
Sklearn est la bibliotheque que nous allons utlisés dans ce projet pour implémenté les modèles de machine learning.

### Description de la base de données

Cette base de données  regroupe les informations récuillies lors d'une enquête. Des candidats, ayant des compétences dans different domaine de l'informatique ont été noté dans ces domaines. Dans chaque domaine on leur attribue les appreciation en fonction de la maitrise de cette domaine. Pour une domaine donnée, les appréciations peuvent être "Non interessé", "debutant" ansi de suite jusqu'a excellent. La base de donnée originalle contient ces notation sous forme de chaine de caractère, elle á été transformer après par en de nombre numérique. Maintenant que la base de données á été explique, avant d'aller plus loin, commençons par l'importée dans notre espace de travail. Notre base de donnée nommée "IT-carrer-proficiency" est déjà disponible sur le plateforme de kaggle, il suffit, de l'intégrée dans le projet en fournissant le chemin d'accès que kaggle offre.

In [2]:
raw_data = pd.read_csv("/kaggle/input/it-carrer-proficiency/IT Career Proficiency Dataset/dataset9000.csv")
proficiency_data = pd.read_csv("/kaggle/input/it-carrer-proficiency/IT Career Proficiency Dataset/dataset9000.data")

In [3]:
raw_data.shape

(9179, 18)

In [4]:
proficiency_data.iloc[:,-1]

0       Database Administrator
1       Database Administrator
2       Database Administrator
3       Database Administrator
4       Database Administrator
                 ...          
9174         Graphics Designer
9175         Graphics Designer
9176         Graphics Designer
9177         Graphics Designer
9178         Graphics Designer
Name: Database Administrator, Length: 9179, dtype: object

In [5]:
target = pd.unique(proficiency_data.iloc[:,-1])

La base de données comporte 9179 lignes, c'est a dire les, 9179 observations, sur 18 colonnes. La dernière colonne qui est la colonne cible est quant à elle toujours sous forme textuelle. Nous allons premièrement la transformer en numerique

In [6]:
annotation_dict = {}
j = 1
for i in target:
    annotation_dict[i] = j
    j=j+1
for index, row in proficiency_data.iterrows():
    proficiency_data.loc[index, "Target"] = annotation_dict[row.iloc[-1]]

In [7]:
annotation_dict

{'Database Administrator': 1,
 'Hardware Engineer': 2,
 'Application Support Engineer': 3,
 'Cyber Security Specialist': 4,
 'Networking Engineer': 5,
 'Software Developer': 6,
 'API Integration Specialist': 7,
 'Project Manager': 8,
 'Information Security Specialist': 9,
 'Technical Writer': 10,
 'AI ML Specialist': 11,
 'Software Tester': 12,
 'Business Analyst': 13,
 'Customer Service Executive': 14,
 'Data Scientist': 15,
 'Helpdesk Engineer': 16,
 'Graphics Designer': 17}

### Le type d'apprentisage 

Le problème que nous faisons fasse est clairement un problème d'apprentissage superviser, puisqu'on cherche á savoir dans quelle catégorie de post serait adéquate á un candidat en IT en se basant sur ces compétences technique. Pour zoomé encore plus, ce n'est pas un problème de regrèssion mais plutot un problème de classification, car on cherche á faire une classification.

Commençons d'abord par recupérer nos caractèristique ainsi que notre cible. Entre temps nous avons transoformé notre cible en donnée numerique mais l'ancien chaine reste, supprimer cette colonne de chaine de caractère et recupérons le X qui est les appréciations dans ces différent domaine et le y qui la cible c'est á dire le domaine correspondant.

In [8]:
prepared_data = proficiency_data.drop("Database Administrator", axis=1)

In [9]:
prepared_data

,9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,1.10,1.11,1.12,1.13,1.14,1.15,Target
0,9,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0
1,9,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0
2,9,5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0
3,9,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0
4,9,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9174,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,9,17.0
9175,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,5,9,17.0
9176,1,1,1,1,1,1,1,1,1,1,1,1,1,1,6,6,9,17.0
9177,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,7,9,17.0


In [10]:
X, y = prepared_data.drop(["Target"], axis = 1), prepared_data.iloc[:, -1]

In [11]:
y

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
9174    17.0
9175    17.0
9176    17.0
9177    17.0
9178    17.0
Name: Target, Length: 9179, dtype: float64

### Standardisation

Pour aider notre modèle á ne pas effectué des calcule très grands, on va standardiser toutes nos varible. Les standardisations consiste á retirer la moyenne d'une variable dans cette variable et la diviser pas son écart-type. Ainsi on va rendre cette variable normal avec une moyenne de 0 et un ecart type de 1. La formule est la suivante: $$ x_i = \frac{x - \mu}{\sigma} $$ où <br> $ x $ est la variable <br> $\mu$   la moyenne <br> $ \sigma$ l'ecart-type 

### Le modele

Comme dit plus haut, le problème present est de type de classification multi-classe. Un des algorithmes de classification mult-classe le plus utilisé est l'algorithme de support vecteur machine. Cet algorithme divise l'espece en de classes correspondant aux classe de notre porblème. Le code suivant est une implémentation de l'algorithme de support vector machine. Mais avant cela, division note donnée en données d'entrainement et données de test

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

In [14]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0


### Resultat

Remarquablement, notre model fonctionne bien, a bien appris la relation qui existe entre ces different données. Il a reuisie á apprendre les règles. Le taux de réuisite de notre modèle est de 100 pourcent

## Conclusion

Dans cette travail, nous avons créer un modèle qui pour repondre au question suivant, comment savoir quels post correspond a un candidat en se basant sur ces compétences technique en IT. Après avoir effectué quelque étapes de purification de donnée, le modèle de SVM á été utilisé pour repondre á cette question. Remarquablement, le modèle a eu un taux de réussite de 100, il a réussis á apprendre les relations entre le compétences et le post ne question